<a href="https://colab.research.google.com/github/Oleg007003/A-model-for-predicting-the-outcome-of-a-football-game/blob/main/%D0%9C%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок с загрузкой данных

In [286]:
!pip install catboost

In [287]:
import numpy as np
from matplotlib import pyplot as plt
import catboost as cb
import keras
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import normalize
import math
from pandas.plotting import table 

In [288]:
data = pd.read_csv('http://assets.codeforces.com/rounds/1522/3056cab52876d3327edafe4771c8589a96cb1a93/train.csv')
del data[data.columns[0]]

In [289]:
data.head(15)

,Division,Time,home_team,away_team,full_time_home_goals,full_time_away_goals,half_time_home_goals,half_time_away_goals,Referee,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_fouls,away_fouls,home_corners,away_corners,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards,home_coef,draw_coef,away_coef
0,0,NaN,152.0,426.0,0.0,0.0,0.0,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.627690,3.577721,2.198600
1,0,NaN,216.0,341.0,2.0,2.0,0.0,1.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.608321,4.251999,6.533805
2,0,NaN,223.0,119.0,2.0,2.0,2.0,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.166811,3.486596,3.821467
3,0,NaN,68.0,137.0,2.0,1.0,0.0,1.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.259244,3.592401,3.460103
4,1,NaN,425.0,225.0,2.0,2.0,1.0,0.0,-1,13.0,11.0,5.0,7.0,18.0,15.0,4.0,10.0,3.0,4.0,1.0,0.0,2.424278,3.414191,3.282876
5,1,NaN,393.0,300.0,0.0,0.0,0.0,0.0,-1,6.0,6.0,1.0,2.0,9.0,13.0,6.0,8.0,1.0,0.0,0.0,0.0,2.787163,3.375001,2.817568
6,1,NaN,239.0,140.0,0.0,0.0,0.0,0.0,-1,16.0,4.0,8.0,1.0,9.0,16.0,6.0,1.0,0.0,2.0,0.0,0.0,2.080239,3.582070,3.998117
7,1,NaN,211.0,424.0,2.0,0.0,1.0,0.0,-1,18.0,5.0,7.0,3.0,12.0,10.0,6.0,6.0,2.0,1.0,0.0,1.0,2.099892,3.449822,4.099789
8,1,NaN,99.0,5.0,1.0,0.0,0.0,0.0,-1,9.0,5.0,5.0,2.0,5.0,9.0,6.0,4.0,0.0,1.0,0.0,0.0,1.312957,5.807312,13.129574
9,0,NaN,303.0,382.0,1.0,0.0,1.0,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.528475,4.476952,7.553624


In [290]:
for i in range(9, 21):
  name = data.columns[i]
  data[name] = data[name].fillna(data[name].mean())

In [291]:
data['is_draw'] = (data['full_time_home_goals'] == data['full_time_away_goals']) + 0
data['is_win'] = (data['full_time_home_goals'] > data['full_time_away_goals']) + 0
data['is_lose'] = (data['full_time_home_goals'] < data['full_time_away_goals']) + 0
data['result'] = data['is_win'] - data['is_lose']

In [292]:
#data = data[data['Division'] < 3]
data.shape

(39992, 28)

In [293]:
y = data['result'].copy()
X = data.drop(['is_win', 'is_draw', 'is_lose', 'result', 'full_time_home_goals', 'full_time_away_goals', 'half_time_home_goals', 'half_time_away_goals'], axis = 1).copy()
Priznak = data.copy()
del Priznak['result']

In [294]:
comands_home = data['home_team'].unique()
comands_away = data['away_team'].unique()

In [465]:
tX = Priznak.copy()
tX['ans'] = y
cr = tX.corr()
cr

,Division,Time,home_team,away_team,full_time_home_goals,full_time_away_goals,half_time_home_goals,half_time_away_goals,Referee,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_fouls,away_fouls,home_corners,away_corners,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards,home_coef,draw_coef,away_coef,is_draw,is_win,is_lose,ans
Division,1.000000,NaN,0.047133,0.049211,-0.013254,-0.039935,-0.003538,-0.027301,-0.180112,0.030268,0.015708,-0.006655,-0.015124,0.114350,0.105286,-0.019299,-0.037350,0.059531,0.046111,0.024133,0.028889,0.000377,-0.031675,0.064134,0.008757,0.012532,-0.022216,0.019352
Time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
home_team,0.047133,NaN,1.000000,-0.018740,-0.011113,0.009119,-0.010328,0.009665,-0.021278,-0.006355,0.021754,-0.011875,0.014998,0.019853,0.004237,0.000222,0.008517,0.009666,0.002308,0.007390,0.011710,-0.000958,-0.041586,-0.033689,0.002370,-0.009613,0.008204,-0.010083
away_team,0.049211,NaN,-0.018740,1.000000,0.008494,-0.002596,0.004354,0.002383,-0.022919,0.012554,0.000867,0.010880,0.005359,-0.007764,0.028218,0.007086,0.005400,0.001551,0.016070,-0.003301,0.010114,-0.038855,-0.015406,0.005382,0.010403,0.005952,-0.016623,0.012463
full_time_home_goals,-0.013254,NaN,-0.011113,0.008494,1.000000,-0.040847,0.678939,-0.013310,-0.019022,0.211962,-0.050846,0.416313,-0.059738,-0.028869,-0.023228,-0.006077,-0.017261,-0.066290,0.006311,-0.060390,0.079200,-0.205403,0.222806,0.286948,-0.240103,0.634570,-0.460177,0.621739
full_time_away_goals,-0.039935,NaN,0.009119,-0.002596,-0.040847,1.000000,-0.025540,0.675955,0.013541,-0.051242,0.219218,-0.054348,0.412652,-0.005005,-0.022499,-0.019999,0.012326,0.058670,-0.020490,0.097551,-0.058282,0.253489,-0.039695,-0.187996,-0.093152,-0.488561,0.624656,-0.624363
half_time_home_goals,-0.003538,NaN,-0.010328,0.004354,0.678939,-0.025540,1.000000,-0.024091,-0.009321,0.087486,0.005083,0.254720,-0.025105,0.007845,-0.017631,-0.060364,0.037997,-0.048861,-0.003346,-0.031259,0.040562,-0.138930,0.154180,0.197018,-0.156580,0.420406,-0.307291,0.413210
half_time_away_goals,-0.027301,NaN,0.009665,0.002383,-0.013310,0.675955,-0.024091,1.000000,0.006339,0.009432,0.101428,-0.010810,0.258338,-0.002822,0.004966,0.033033,-0.036777,0.048193,-0.003769,0.050189,-0.018936,0.176722,-0.024129,-0.127232,-0.059498,-0.320861,0.408608,-0.409168
Referee,-0.180112,NaN,-0.021278,-0.022919,-0.019022,0.013541,-0.009321,0.006339,1.000000,-0.076379,-0.034470,-0.065282,-0.024241,-0.188915,-0.174333,0.004185,0.049867,-0.134403,-0.121851,-0.022890,-0.035472,-0.012706,-0.047371,-0.087907,-0.008059,-0.018734,0.028316,-0.026293
home_shots,0.030268,NaN,-0.006355,0.012554,0.211962,-0.051242,0.087486,0.009432,-0.076379,1.000000,-0.167672,0.640322,-0.151170,-0.078802,0.028039,0.423836,-0.232736,-0.081064,0.050278,-0.104704,0.101868,-0.221339,0.210587,0.297839,-0.029578,0.128745,-0.111973,0.136169


# Блок про игру друг с другом

In [469]:
results_games_shots = np.zeros((495,495))
results_games_itog = np.zeros((495,495))
games = Priznak[['home_team', 'away_team', 'full_time_home_goals', 'full_time_away_goals']]
for index, row in games.iterrows():
  hteam = int(row['home_team'])
  ateam = int(row['away_team'])
  hshots = int(row['full_time_home_goals'])
  ashots = int(row['full_time_away_goals'])
  results_games_shots[hteam][ateam] += hshots - ashots
  results_games_shots[ateam][hteam] += ashots - hshots
  results_games_itog[hteam][ateam] += (hshots > ashots) * 1
  results_games_itog[hteam][ateam] += (hshots < ashots) * 1

# Блок про личные характеристики

In [297]:
name_home_priznak = ['home_team', 'Division', 'full_time_home_goals', 'half_time_home_goals', 'home_shots', 'home_shots_on_target', 'home_fouls', 'home_corners', 'home_yellow_cards', 'home_red_cards', 'home_coef']
name_away_priznak = ['away_team', 'full_time_away_goals', 'half_time_away_goals', 'away_shots', 'away_shots_on_target', 'away_fouls', 'away_corners', 'away_yellow_cards', 'away_red_cards', 'away_coef']
name_draw_priznak = ['draw_coef']

In [298]:
home_priznak = Priznak[name_home_priznak]
away_priznak = Priznak[name_away_priznak]
draw_priznak = Priznak[name_draw_priznak]

In [299]:
minPlayerId = min(min(data['home_team']), min(data['away_team'])) # == 1
maxPlayerId = max(max(data['home_team']), max(data['away_team'])) # == 494

players = pd.DataFrame()
players['id'] = range(0, int(maxPlayerId) + 1)
lst_home = []
lst_away = []
for i in range(0, int(maxPlayerId) + 1):
  lst_home.append(Priznak[Priznak['home_team'] == i].shape[0])
  lst_away.append(Priznak[Priznak['away_team'] == i].shape[0])
players['games_home'] = pd.DataFrame(lst_home)
players['games_away'] = pd.DataFrame(lst_away)

In [300]:
players

,id,games_home,games_away
0,0,0,0
1,1,49,46
2,2,136,135
3,3,100,100
4,4,50,51
...,...,...,...
490,490,12,14
491,491,136,135
492,492,92,92
493,493,117,118


In [301]:
maxPlayerId

494.0

In [302]:
p_home = pd.DataFrame()
for cols in range(1, len(name_home_priznak)):
  name = name_home_priznak[cols]
  lst = [0]
  for i in range(int(minPlayerId), int(maxPlayerId) + 1):
    lst.append(home_priznak[home_priznak['home_team'] == i][name].sum())
  p_home['players_'+name] = lst

In [303]:
p_away = pd.DataFrame()
for cols in range(1, len(name_away_priznak)):
  name = name_away_priznak[cols]
  lst = [0]
  for i in range(int(minPlayerId), int(maxPlayerId) + 1):
    lst.append(away_priznak[away_priznak['away_team'] == i][name].sum())
  p_away['players_'+name] = lst

In [304]:
players = players.drop(0)
p_home = p_home.drop(0)
p_away = p_away.drop(0)

In [305]:
p_home

,players_Division,players_full_time_home_goals,players_half_time_home_goals,players_home_shots,players_home_shots_on_target,players_home_fouls,players_home_corners,players_home_yellow_cards,players_home_red_cards,players_home_coef
1,784,89.0,41.0,654.604148,270.987786,594.910310,309.215632,82.336230,6.329658,97.465730
2,1406,174.0,79.0,1512.000000,606.000000,1547.000000,837.000000,182.000000,7.000000,350.692666
3,200,197.0,89.0,1382.532011,529.358308,1203.846245,612.925978,157.724187,10.171571,205.464595
4,0,66.0,22.0,644.364292,244.856136,607.051336,287.455939,83.764397,4.268793,139.517610
5,110,161.0,75.0,1280.000000,539.000000,1242.000000,746.000000,136.000000,8.000000,236.691254
...,...,...,...,...,...,...,...,...,...,...
490,60,16.0,7.0,101.000000,47.000000,189.000000,67.000000,25.000000,0.000000,40.520500
491,1312,147.0,69.0,1364.000000,557.000000,1591.000000,692.000000,188.000000,13.000000,416.975770
492,1012,97.0,53.0,977.000000,436.000000,896.000000,578.000000,102.000000,7.000000,249.108100
493,816,156.0,80.0,1496.757439,554.564063,1525.564191,662.427740,231.373612,16.000820,269.017019


In [306]:
p_home[p_home.columns[0]] //= players['games_home']
for i in range(1, len(p_home.columns)):
  p_home[p_home.columns[i]] /= players['games_home']

In [307]:
for i in range(0, len(p_away.columns)):
  p_away[p_away.columns[i]] /= players['games_away']

In [308]:
p_away.reset_index(drop=True, inplace=True)
p_home.reset_index(drop=True, inplace=True)
players.reset_index(drop=True, inplace=True)
a = pd.concat([p_away, p_home], axis = 1)
a.reset_index(drop=True, inplace=True)
players = pd.concat([players, a], axis = 1)

In [309]:
players

,id,games_home,games_away,players_full_time_away_goals,players_half_time_away_goals,players_away_shots,players_away_shots_on_target,players_away_fouls,players_away_corners,players_away_yellow_cards,players_away_red_cards,players_away_coef,players_Division,players_full_time_home_goals,players_half_time_home_goals,players_home_shots,players_home_shots_on_target,players_home_fouls,players_home_corners,players_home_yellow_cards,players_home_red_cards,players_home_coef
0,1,49,46,0.956522,0.478261,10.682363,4.304612,12.679677,4.834008,2.286677,0.092639,3.542901,16.0,1.816327,0.836735,13.359268,5.530363,12.141027,6.310523,1.680331,0.129177,1.989097
1,2,136,135,1.037037,0.377778,9.651852,3.903704,11.718519,5.755556,1.444444,0.096296,3.708734,10.0,1.279412,0.580882,11.117647,4.455882,11.375000,6.154412,1.338235,0.051471,2.578623
2,3,100,100,1.670000,0.730000,11.580175,4.400275,12.740929,4.913137,1.971546,0.096746,3.378254,2.0,1.970000,0.890000,13.825320,5.293583,12.038462,6.129260,1.577242,0.101716,2.054646
3,4,50,51,0.823529,0.274510,10.524018,4.000328,12.679677,4.658497,1.978031,0.115173,4.894284,0.0,1.320000,0.440000,12.887286,4.897123,12.141027,5.749119,1.675288,0.085376,2.790352
4,5,110,113,1.575221,0.743363,10.442478,4.477876,11.221239,5.663717,1.486726,0.079646,2.842445,1.0,1.463636,0.681818,11.636364,4.900000,11.290909,6.781818,1.236364,0.072727,2.151739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,490,12,14,0.857143,0.500000,6.714286,3.642857,12.857143,4.000000,2.214286,0.071429,4.863078,5.0,1.333333,0.583333,8.416667,3.916667,15.750000,5.583333,2.083333,0.000000,3.376708
490,491,136,135,1.111111,0.466667,9.762963,3.792593,12.303704,4.392593,1.614815,0.088889,4.739822,9.0,1.080882,0.507353,10.029412,4.095588,11.698529,5.088235,1.382353,0.095588,3.065998
491,492,92,92,1.108696,0.456522,8.597826,3.684783,10.630435,4.956522,1.445652,0.097826,3.897203,11.0,1.054348,0.576087,10.619565,4.739130,9.739130,6.282609,1.108696,0.076087,2.707697
492,493,117,118,1.110169,0.576271,10.681741,3.907005,13.398761,4.528290,2.315428,0.138382,4.362666,6.0,1.333333,0.683761,12.792799,4.739864,13.039010,5.661776,1.977552,0.136759,2.299291


In [310]:
copy_player = players.copy()
del copy_player['id']

In [311]:
copy_player.columns

Index(['games_home', 'games_away', 'players_full_time_away_goals',
       'players_half_time_away_goals', 'players_away_shots',
       'players_away_shots_on_target', 'players_away_fouls',
       'players_away_corners', 'players_away_yellow_cards',
       'players_away_red_cards', 'players_away_coef', 'players_Division',
       'players_full_time_home_goals', 'players_half_time_home_goals',
       'players_home_shots', 'players_home_shots_on_target',
       'players_home_fouls', 'players_home_corners',
       'players_home_yellow_cards', 'players_home_red_cards',
       'players_home_coef'],
      dtype='object')

In [312]:
names_home = ['players_full_time_home_goals', 'players_half_time_home_goals',
       'players_home_shots', 'players_home_shots_on_target',
       'players_home_fouls', 'players_home_corners',
       'players_home_yellow_cards', 'players_home_red_cards',
       'players_home_coef']
names_away = ['players_full_time_away_goals',
       'players_half_time_away_goals', 'players_away_shots',
       'players_away_shots_on_target', 'players_away_fouls',
       'players_away_corners', 'players_away_yellow_cards',
       'players_away_red_cards', 'players_away_coef']

In [313]:
df_h = pd.DataFrame()
df_a = pd.DataFrame()
df_h[names_home] = np.nan
df_a[names_away] = np.nan

In [314]:
for index, row in Priznak.iterrows():
  home = (players[players['id'] == int(row['home_team'])][names_home])
  away = (players[players['id'] == int(row['away_team'])][names_away])
  df_h = df_h.append(home, ignore_index=True)
  df_a = df_a.append(away, ignore_index=True)

In [600]:
df_h

,players_full_time_home_goals,players_half_time_home_goals,players_home_shots,players_home_shots_on_target,players_home_fouls,players_home_corners,players_home_yellow_cards,players_home_red_cards,players_home_coef
0,1.228916,0.626506,12.811270,4.867522,12.223974,5.601688,1.697826,0.081995,2.962474
1,1.390000,0.550000,11.320839,4.302511,14.353975,4.767350,1.514799,0.054514,2.934653
2,1.357143,0.561224,13.881268,4.733226,13.296442,5.637306,1.732290,0.084375,2.667181
3,1.464646,0.585859,12.964970,4.842003,12.311578,5.706323,1.704285,0.082541,2.413541
4,1.287037,0.555556,11.613708,4.672269,10.515314,6.349268,1.345227,0.100161,3.303867
...,...,...,...,...,...,...,...,...,...
39987,2.243243,1.117117,17.720721,6.585586,12.855856,6.351351,1.738739,0.045045,1.418327
39988,1.990991,0.900901,15.819820,5.675676,15.054054,6.405405,2.135135,0.144144,2.020884
39989,1.383929,0.651786,12.982143,4.321429,13.803571,4.910714,2.080357,0.151786,2.956302
39990,1.356436,0.643564,12.595720,4.599031,13.087078,5.054159,1.692671,0.068344,2.685224


# Блок про слияние

In [475]:
X_r = X[['Division', 'Time', 'home_team', 'away_team', 'Referee', 'home_coef', 'draw_coef', 'away_coef']]
X_r.reset_index(drop=True, inplace=True)
a = pd.concat([df_h, df_a], axis = 1)
a.reset_index(drop=True, inplace=True)
X_r = pd.concat([X_r, a], axis = 1)

In [472]:
teams_res_shots = np.zeros(X_r.shape[0])
teams_res_itog = np.zeros(X_r.shape[0])
for index, row in X_r.iterrows():
  hteam = int(row['home_team'])
  ateam = int(row['away_team'])
  teams_res_shots[index] = results_games_shots[hteam][ateam]
  teams_res_itog[index] = results_games_itog[hteam][ateam]

In [479]:
X_r['personal_res_shots'] = teams_res_shots
X_r['personal_res_itog'] = teams_res_itog
X_r['strong_personal_res_shots'] = (teams_res_shots > 10) * 1
X_r['weak_personal_shots'] = (teams_res_shots < -10) * 1

In [480]:
normalized_X=(X_r-X_r.mean())/X_r.std()
#tX = normalized_X.copy()
#tX['ans'] = y_train['result']
X_corr = normalized_X.copy()
X_corr['ans'] = y
cr = X_corr.corr()
cr

,Division,Time,home_team,away_team,Referee,home_coef,draw_coef,away_coef,players_full_time_home_goals,players_half_time_home_goals,players_home_shots,players_home_shots_on_target,players_home_fouls,players_home_corners,players_home_yellow_cards,players_home_red_cards,players_home_coef,players_full_time_away_goals,players_half_time_away_goals,players_away_shots,players_away_shots_on_target,players_away_fouls,players_away_corners,players_away_yellow_cards,players_away_red_cards,players_away_coef,personal_res_shots,personal_res_itog,strong_personal_res_shots,weak_personal_shots,ans
Division,1.000000,NaN,0.047133,0.049211,-0.180112,0.000377,-0.031675,0.064134,-0.018811,0.005492,0.091172,0.008035,0.222163,-0.048094,0.152875,0.157459,-0.081042,-0.136534,-0.125468,0.061499,-0.030690,0.214148,-0.143394,0.129890,0.197658,0.081726,0.002527,-0.143220,-0.014946,-0.017551,0.019352
Time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
home_team,0.047133,NaN,1.000000,-0.018740,-0.021278,-0.000958,-0.041586,-0.033689,-0.040855,-0.050490,-0.016027,-0.039641,0.045780,0.001001,0.031023,0.057583,-0.002763,-0.000746,-0.000891,0.024398,0.015681,-0.004041,0.006379,-0.004472,-0.002383,0.012292,-0.044573,0.006148,-0.054646,0.021350,-0.010083
away_team,0.049211,NaN,-0.018740,1.000000,-0.022919,-0.038855,-0.015406,0.005382,0.008638,0.008846,0.026559,0.018150,-0.002169,0.016969,-0.001001,-0.002869,-0.003483,-0.010249,0.012153,0.002515,0.020000,0.069440,0.026295,0.055536,0.074573,0.014426,0.045153,-0.000886,0.018753,-0.055830,0.012463
Referee,-0.180112,NaN,-0.021278,-0.022919,1.000000,-0.012706,-0.047371,-0.087907,-0.080627,-0.064512,-0.213832,-0.069764,-0.469287,0.039771,-0.426657,-0.216775,-0.022437,0.035095,0.021604,-0.167402,0.006152,-0.450196,0.222383,-0.414337,-0.224359,-0.212899,0.000650,0.012630,-0.048300,-0.046782,-0.026293
home_coef,0.000377,NaN,-0.000958,-0.038855,-0.012706,1.000000,0.074698,-0.405132,-0.239771,-0.218849,-0.161247,-0.193109,0.062958,-0.185418,0.089591,0.084004,0.346951,0.575282,0.526800,0.424776,0.492890,-0.088228,0.353828,-0.035167,-0.104492,-0.471031,-0.527433,0.052491,-0.165160,0.467267,-0.278270
draw_coef,-0.031675,NaN,-0.041586,-0.015406,-0.047371,0.074698,1.000000,0.830170,0.620056,0.587905,0.423266,0.523376,-0.059885,0.371620,-0.067106,-0.108230,-0.363421,0.112128,0.102253,0.089555,0.069245,0.030884,0.000202,0.077023,0.001245,0.103143,0.296261,0.160188,0.444059,0.081981,0.130303
away_coef,0.064134,NaN,-0.033689,0.005382,-0.087907,-0.405132,0.830170,1.000000,0.632511,0.596788,0.450558,0.534446,-0.031679,0.407108,-0.052977,-0.111630,-0.497692,-0.260311,-0.238253,-0.150795,-0.220571,0.115163,-0.217998,0.129233,0.087866,0.373107,0.530259,0.069534,0.472490,-0.166948,0.264587
players_full_time_home_goals,-0.018811,NaN,-0.040855,0.008638,-0.080627,-0.239771,0.620056,0.632511,1.000000,0.922391,0.618011,0.738483,-0.068803,0.535530,-0.108516,-0.186983,-0.691081,0.093328,0.094829,0.094927,0.029058,0.094010,-0.034479,0.093685,0.031236,0.066231,0.446943,0.274528,0.497938,-0.051723,0.192250
players_half_time_home_goals,0.005492,NaN,-0.050490,0.008846,-0.064512,-0.218849,0.587905,0.596788,0.922391,1.000000,0.577863,0.700104,-0.050035,0.505667,-0.101754,-0.179526,-0.630780,0.085547,0.086609,0.088088,0.027904,0.094871,-0.027134,0.096115,0.033771,0.063917,0.415643,0.253101,0.468136,-0.047343,0.173425


In [481]:
print(X_r.dtypes)
X_r['Division'] = X_r['Division'].apply(str)
X_r['home_team'] = X_r['home_team'].apply(str)
X_r['away_team'] = X_r['away_team'].apply(str)
X_r['Referee'] = X_r['Referee'].apply(str)

Division                          int64
Time                            float64
home_team                       float64
away_team                       float64
Referee                           int64
home_coef                       float64
draw_coef                       float64
away_coef                       float64
players_full_time_home_goals    float64
players_half_time_home_goals    float64
players_home_shots              float64
players_home_shots_on_target    float64
players_home_fouls              float64
players_home_corners            float64
players_home_yellow_cards       float64
players_home_red_cards          float64
players_home_coef               float64
players_full_time_away_goals    float64
players_half_time_away_goals    float64
players_away_shots              float64
players_away_shots_on_target    float64
players_away_fouls              float64
players_away_corners            float64
players_away_yellow_cards       float64
players_away_red_cards          float64


# Блок про модели

In [575]:
y.shape

(39992,)

In [576]:
X_r.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)
data_tmp = pd.concat([X_r, y], axis = 1)
data_tmp['Division'] = data_tmp['Division'].apply(int)
data_tmp = data_tmp[(data_tmp['Division'] > 9) & (data_tmp['Division'] < 12)]
data_tmp, data_test = train_test_split(data_tmp, test_size = 0.15, random_state= 13, shuffle = False)

In [577]:
data_tmp

,Division,Time,home_team,away_team,Referee,home_coef,draw_coef,away_coef,players_full_time_home_goals,players_half_time_home_goals,players_home_shots,players_home_shots_on_target,players_home_fouls,players_home_corners,players_home_yellow_cards,players_home_red_cards,players_home_coef,players_full_time_away_goals,players_half_time_away_goals,players_away_shots,players_away_shots_on_target,players_away_fouls,players_away_corners,players_away_yellow_cards,players_away_red_cards,players_away_coef,personal_res_shots,personal_res_itog,strong_personal_res_shots,weak_personal_shots,result
65,10,NaN,33.0,436.0,0,1.448041,4.786579,9.050254,2.133929,0.919643,17.035714,7.098214,9.767857,7.276786,1.285714,0.071429,1.677148,1.042735,0.470085,9.649573,3.632479,11.512821,3.957265,2.000000,0.119658,7.105781,15.0,3.0,1,0,0
71,10,NaN,479.0,264.0,1,4.370737,3.584204,1.991224,1.300885,0.566372,12.415929,4.362832,10.734513,5.318584,1.619469,0.035398,3.383553,1.858407,0.867257,14.619469,5.884956,10.716814,5.911504,1.805310,0.044248,2.417817,-1.0,2.0,0,0,1
72,10,NaN,181.0,318.0,2,1.852456,3.815059,4.806373,1.604651,0.697674,15.062016,5.348837,10.255814,6.224806,1.449612,0.100775,2.671934,1.048000,0.448000,11.056000,3.896000,11.952000,4.376000,1.888000,0.120000,4.608339,10.0,3.0,0,0,1
82,10,NaN,310.0,444.0,3,2.708460,3.480873,2.828836,1.482759,0.577586,14.525862,5.215517,11.146552,6.172414,1.663793,0.068966,2.888161,1.734513,0.663717,14.398230,5.690265,11.194690,5.831858,1.867257,0.070796,2.846053,-2.0,5.0,0,0,1
83,10,NaN,369.0,437.0,4,2.097615,3.525992,4.005447,1.364341,0.488372,14.961240,5.186047,12.023256,5.751938,1.403101,0.062016,2.825678,0.982143,0.383929,9.857143,3.589286,10.598214,4.080357,1.410714,0.053571,6.108396,-10.0,1.0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33857,10,NaN,310.0,444.0,9,5.463225,3.973254,1.738299,1.482759,0.577586,14.525862,5.215517,11.146552,6.172414,1.663793,0.068966,2.888161,1.734513,0.663717,14.398230,5.690265,11.194690,5.831858,1.867257,0.070796,2.846053,-2.0,5.0,0,0,-1
33921,11,NaN,7.0,282.0,83,2.942489,3.466494,2.620025,1.558824,0.691176,11.654412,4.985294,9.286765,5.779412,1.242647,0.073529,2.593540,0.955752,0.424779,9.415929,3.716814,12.840708,4.610619,2.088496,0.132743,3.417948,7.0,3.0,0,0,1
33935,11,NaN,122.0,92.0,46,3.374652,3.444130,2.362256,1.214815,0.518519,11.081481,4.503704,11.088889,4.962963,1.585185,0.111111,3.022039,1.112360,0.404494,9.393258,3.629213,10.865169,4.696629,1.483146,0.044944,3.434567,-3.0,3.0,0,0,-1
33945,11,NaN,95.0,108.0,138,2.054692,3.738537,3.908926,1.430657,0.649635,11.810219,4.708029,10.102190,5.963504,1.240876,0.065693,2.673860,1.078947,0.526316,9.780702,3.938596,11.956140,4.815789,1.684211,0.035088,3.418343,4.0,2.0,0,0,1


In [578]:
yt = data_tmp['result'].copy()
del data_tmp['result']
Xt_r = data_tmp

In [579]:
Xt_r['Division'] = Xt_r['Division'].apply(str)
Xt_r['home_team'] = Xt_r['home_team'].apply(str)
Xt_r['away_team'] = Xt_r['away_team'].apply(str)
Xt_r.dtypes
Xt_r.reset_index(drop=True, inplace=True)
yt.reset_index(drop=True, inplace=True)

In [580]:
yt.isna().sum()

0

In [581]:
Xt_r.isna().sum()

Division                           0
Time                            4673
home_team                          0
away_team                          0
Referee                            0
home_coef                          0
draw_coef                          0
away_coef                          0
players_full_time_home_goals       0
players_half_time_home_goals       0
players_home_shots                 0
players_home_shots_on_target       0
players_home_fouls                 0
players_home_corners               0
players_home_yellow_cards          0
players_home_red_cards             0
players_home_coef                  0
players_full_time_away_goals       0
players_half_time_away_goals       0
players_away_shots                 0
players_away_shots_on_target       0
players_away_fouls                 0
players_away_corners               0
players_away_yellow_cards          0
players_away_red_cards             0
players_away_coef                  0
personal_res_shots                 0
p

In [582]:
cat = cb.CatBoostClassifier(cat_features=[0, 2, 3, 4])
grid = {'learning_rate': [0.03, 0.05, 0.6, 0.075, 0.875, 0.1, 0.11, 0.125, 0.5, 0.8, 1],
        'depth': [3, 4, 5, 6, 7, 8, 9, 10],
        'l2_leaf_reg': [0.1, 0.5, 1, 1.6, 1.8, 2, 2.2, 2.4, 2.7, 3, 3.3, 3.5, 4, 5, 7, 9, 11],
        'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide'],
        'auto_class_weights': ['None', 'Balanced', 'SqrtBalanced'],
        'od_pval': [0, 0.0000000001, 0.0000001, 0.00001, 0.0001, 0.001, 0.01],
        'posterior_sampling': [True, False]}
cat.randomized_search(grid, Xt_r, yt, n_iter = 3, cv = 4)
#cat = cat.fit(normalized_X,y_train['result'], verbose=False)

0:	learn: 0.9720999	test: 0.9977240	best: 0.9977240 (0)	total: 121ms	remaining: 2m
1:	learn: 0.9264685	test: 0.9657193	best: 0.9657193 (1)	total: 236ms	remaining: 1m 57s
2:	learn: 0.9025328	test: 0.9513114	best: 0.9513114 (2)	total: 346ms	remaining: 1m 55s
3:	learn: 0.8843151	test: 0.9451848	best: 0.9451848 (3)	total: 442ms	remaining: 1m 50s
4:	learn: 0.8611995	test: 0.9453732	best: 0.9451848 (3)	total: 561ms	remaining: 1m 51s
5:	learn: 0.8411650	test: 0.9400043	best: 0.9400043 (5)	total: 691ms	remaining: 1m 54s
6:	learn: 0.8248193	test: 0.9339971	best: 0.9339971 (6)	total: 802ms	remaining: 1m 53s
7:	learn: 0.8038811	test: 0.9261773	best: 0.9261773 (7)	total: 920ms	remaining: 1m 54s
8:	learn: 0.7947700	test: 0.9271328	best: 0.9261773 (7)	total: 1.03s	remaining: 1m 53s
9:	learn: 0.7827303	test: 0.9215708	best: 0.9215708 (9)	total: 1.13s	remaining: 1m 51s
10:	learn: 0.7703360	test: 0.9275103	best: 0.9215708 (9)	total: 1.26s	remaining: 1m 53s
11:	learn: 0.7600540	test: 0.9291839	best: 0.9

{'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
               47,
               48,
               49,
             

In [583]:
Xt_r

,Division,Time,home_team,away_team,Referee,home_coef,draw_coef,away_coef,players_full_time_home_goals,players_half_time_home_goals,players_home_shots,players_home_shots_on_target,players_home_fouls,players_home_corners,players_home_yellow_cards,players_home_red_cards,players_home_coef,players_full_time_away_goals,players_half_time_away_goals,players_away_shots,players_away_shots_on_target,players_away_fouls,players_away_corners,players_away_yellow_cards,players_away_red_cards,players_away_coef,personal_res_shots,personal_res_itog,strong_personal_res_shots,weak_personal_shots
0,10,NaN,33.0,436.0,0,1.448041,4.786579,9.050254,2.133929,0.919643,17.035714,7.098214,9.767857,7.276786,1.285714,0.071429,1.677148,1.042735,0.470085,9.649573,3.632479,11.512821,3.957265,2.000000,0.119658,7.105781,15.0,3.0,1,0
1,10,NaN,479.0,264.0,1,4.370737,3.584204,1.991224,1.300885,0.566372,12.415929,4.362832,10.734513,5.318584,1.619469,0.035398,3.383553,1.858407,0.867257,14.619469,5.884956,10.716814,5.911504,1.805310,0.044248,2.417817,-1.0,2.0,0,0
2,10,NaN,181.0,318.0,2,1.852456,3.815059,4.806373,1.604651,0.697674,15.062016,5.348837,10.255814,6.224806,1.449612,0.100775,2.671934,1.048000,0.448000,11.056000,3.896000,11.952000,4.376000,1.888000,0.120000,4.608339,10.0,3.0,0,0
3,10,NaN,310.0,444.0,3,2.708460,3.480873,2.828836,1.482759,0.577586,14.525862,5.215517,11.146552,6.172414,1.663793,0.068966,2.888161,1.734513,0.663717,14.398230,5.690265,11.194690,5.831858,1.867257,0.070796,2.846053,-2.0,5.0,0,0
4,10,NaN,369.0,437.0,4,2.097615,3.525992,4.005447,1.364341,0.488372,14.961240,5.186047,12.023256,5.751938,1.403101,0.062016,2.825678,0.982143,0.383929,9.857143,3.589286,10.598214,4.080357,1.410714,0.053571,6.108396,-10.0,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4668,10,NaN,310.0,444.0,9,5.463225,3.973254,1.738299,1.482759,0.577586,14.525862,5.215517,11.146552,6.172414,1.663793,0.068966,2.888161,1.734513,0.663717,14.398230,5.690265,11.194690,5.831858,1.867257,0.070796,2.846053,-2.0,5.0,0,0
4669,11,NaN,7.0,282.0,83,2.942489,3.466494,2.620025,1.558824,0.691176,11.654412,4.985294,9.286765,5.779412,1.242647,0.073529,2.593540,0.955752,0.424779,9.415929,3.716814,12.840708,4.610619,2.088496,0.132743,3.417948,7.0,3.0,0,0
4670,11,NaN,122.0,92.0,46,3.374652,3.444130,2.362256,1.214815,0.518519,11.081481,4.503704,11.088889,4.962963,1.585185,0.111111,3.022039,1.112360,0.404494,9.393258,3.629213,10.865169,4.696629,1.483146,0.044944,3.434567,-3.0,3.0,0,0
4671,11,NaN,95.0,108.0,138,2.054692,3.738537,3.908926,1.430657,0.649635,11.810219,4.708029,10.102190,5.963504,1.240876,0.065693,2.673860,1.078947,0.526316,9.780702,3.938596,11.956140,4.815789,1.684211,0.035088,3.418343,4.0,2.0,0,0


In [584]:
print(cat.score(Xt_r,yt))

0.6687352878236679


# Блок про подсчёт итогов

In [585]:
predict = cat.predict_proba(Xt_r)
real_result = yt.to_numpy()
coefs = Xt_r[['away_coef', 'draw_coef', 'home_coef']].to_numpy()
yLose = (real_result == -1) + 0
errorLose = abs(predict[:, 0] - yLose)
yDraw = (real_result == 0) * 1
errorDraw = abs(predict[:, 1] - yDraw)
yWin = (real_result == 1) * 1
errorWin = abs(predict[:, 2] - yWin)
print(errorLose.mean())
print(errorDraw.mean())
print(errorWin.mean())

0.29755313903974223
0.2937576364274337
0.33134915111689606


In [586]:
coefs

array([[9.05025376, 4.78657866, 1.4480406 ],
       [1.99122418, 3.58420352, 4.37073707],
       [4.80637338, 3.81505887, 1.85245641],
       ...,
       [2.3622564 , 3.44413013, 3.374652  ],
       [3.90892617, 3.73853708, 2.05469196],
       [3.0293058 , 3.52409241, 2.5244215 ]])

In [587]:
moneys = 0
for i, row in Xt_r.iterrows():
  hteam = int(float((row['home_team'])))
  #print(hteam)
  ateam = int(float((row['away_team'])))
  if predict[i][0] > 0.48:
    moneys -= 1
    if real_result[i] == -1:
      moneys += coefs[i][0]
  elif predict[i][1] > 0.48:
    moneys -= 1
    if real_result[i] == 0:
      moneys += coefs[i][1]
  elif predict[i][2] > 0.48:
    moneys -= 1
    if real_result[i] == 1:
      moneys += coefs[i][2]
print(moneys)

3410.2223597413977


In [588]:
Xt_r.shape

(4673, 30)

In [589]:
Xt_r

,Division,Time,home_team,away_team,Referee,home_coef,draw_coef,away_coef,players_full_time_home_goals,players_half_time_home_goals,players_home_shots,players_home_shots_on_target,players_home_fouls,players_home_corners,players_home_yellow_cards,players_home_red_cards,players_home_coef,players_full_time_away_goals,players_half_time_away_goals,players_away_shots,players_away_shots_on_target,players_away_fouls,players_away_corners,players_away_yellow_cards,players_away_red_cards,players_away_coef,personal_res_shots,personal_res_itog,strong_personal_res_shots,weak_personal_shots
0,10,NaN,33.0,436.0,0,1.448041,4.786579,9.050254,2.133929,0.919643,17.035714,7.098214,9.767857,7.276786,1.285714,0.071429,1.677148,1.042735,0.470085,9.649573,3.632479,11.512821,3.957265,2.000000,0.119658,7.105781,15.0,3.0,1,0
1,10,NaN,479.0,264.0,1,4.370737,3.584204,1.991224,1.300885,0.566372,12.415929,4.362832,10.734513,5.318584,1.619469,0.035398,3.383553,1.858407,0.867257,14.619469,5.884956,10.716814,5.911504,1.805310,0.044248,2.417817,-1.0,2.0,0,0
2,10,NaN,181.0,318.0,2,1.852456,3.815059,4.806373,1.604651,0.697674,15.062016,5.348837,10.255814,6.224806,1.449612,0.100775,2.671934,1.048000,0.448000,11.056000,3.896000,11.952000,4.376000,1.888000,0.120000,4.608339,10.0,3.0,0,0
3,10,NaN,310.0,444.0,3,2.708460,3.480873,2.828836,1.482759,0.577586,14.525862,5.215517,11.146552,6.172414,1.663793,0.068966,2.888161,1.734513,0.663717,14.398230,5.690265,11.194690,5.831858,1.867257,0.070796,2.846053,-2.0,5.0,0,0
4,10,NaN,369.0,437.0,4,2.097615,3.525992,4.005447,1.364341,0.488372,14.961240,5.186047,12.023256,5.751938,1.403101,0.062016,2.825678,0.982143,0.383929,9.857143,3.589286,10.598214,4.080357,1.410714,0.053571,6.108396,-10.0,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4668,10,NaN,310.0,444.0,9,5.463225,3.973254,1.738299,1.482759,0.577586,14.525862,5.215517,11.146552,6.172414,1.663793,0.068966,2.888161,1.734513,0.663717,14.398230,5.690265,11.194690,5.831858,1.867257,0.070796,2.846053,-2.0,5.0,0,0
4669,11,NaN,7.0,282.0,83,2.942489,3.466494,2.620025,1.558824,0.691176,11.654412,4.985294,9.286765,5.779412,1.242647,0.073529,2.593540,0.955752,0.424779,9.415929,3.716814,12.840708,4.610619,2.088496,0.132743,3.417948,7.0,3.0,0,0
4670,11,NaN,122.0,92.0,46,3.374652,3.444130,2.362256,1.214815,0.518519,11.081481,4.503704,11.088889,4.962963,1.585185,0.111111,3.022039,1.112360,0.404494,9.393258,3.629213,10.865169,4.696629,1.483146,0.044944,3.434567,-3.0,3.0,0,0
4671,11,NaN,95.0,108.0,138,2.054692,3.738537,3.908926,1.430657,0.649635,11.810219,4.708029,10.102190,5.963504,1.240876,0.065693,2.673860,1.078947,0.526316,9.780702,3.938596,11.956140,4.815789,1.684211,0.035088,3.418343,4.0,2.0,0,0


In [590]:
results_games_shots.tofile('res_games_shots.csv',sep=',',format='%10.5f')
results_games_itog.tofile('res_games_itog.csv',sep=',',format='%10.5f')

# Блок про настоящий подсчёт

In [591]:
ytest = data_test['result'].copy()
del data_test['result']
Xtest_r = data_test

In [592]:
Xtest_r['Division'] = Xtest_r['Division'].apply(str)
Xtest_r['home_team'] = Xtest_r['home_team'].apply(str)
Xtest_r['away_team'] = Xtest_r['away_team'].apply(str)
Xtest_r.dtypes
Xtest_r.reset_index(drop=True, inplace=True)
ytest.reset_index(drop=True, inplace=True)

In [593]:
print(cat.score(Xtest_r,ytest))

0.6666666666666666


In [594]:
predict = cat.predict_proba(Xtest_r)
real_result = ytest.to_numpy()
coefs = Xtest_r[['away_coef', 'draw_coef', 'home_coef']].to_numpy()
yLose = (real_result == -1) + 0
errorLose = abs(predict[:, 0] - yLose)
yDraw = (real_result == 0) * 1
errorDraw = abs(predict[:, 1] - yDraw)
yWin = (real_result == 1) * 1
errorWin = abs(predict[:, 2] - yWin)
print(errorLose.mean())
print(errorDraw.mean())
print(errorWin.mean())

0.30043352446579247
0.2515554331564074
0.3356413844947036


In [595]:
moneys = 0
for i, row in Xtest_r.iterrows():
  hteam = int(float((row['home_team'])))
  ateam = int(float((row['away_team'])))
  if predict[i][0] > 0.48:
    moneys -= 1
    if real_result[i] == -1:
      moneys += coefs[i][0]
  elif predict[i][1] > 0.48:
    moneys -= 1
    if real_result[i] == 0:
      moneys += coefs[i][1]
  elif predict[i][2] > 0.48:
    moneys -= 1
    if real_result[i] == 1:
      moneys += coefs[i][2]
print(moneys, Xtest_r.shape[0])

548.1328707398649 825


In [596]:
cat.save_model('model3-5')

In [599]:
from google.colab import files
files.download('res_games_shots.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>